In [2]:
import numpy as np
#np.random.seed(1234)

In [3]:
def exponential_rng(lam=1.0):  
    """ Generates exponential random number.
    
    Keywords:
        lam (float): the rate parameter, the inverse expectation of the distribution.
    
    Returns:
        exponential random number with rate lam.
    """
    return -np.log(np.random.rand()) / lam

In [4]:
def homogeneous_poisson_process(lam, T):
    arrivals = []
    t = exponential_rng(lam)
    while t <= T:
        arrivals.append(t)
        t += exponential_rng(lam)
    return arrivals

In [5]:
def totalRevenue(choices):
    
    revenueList = [0,1000,900,850,750,700,650,600,500,350]
    revenueList = np.flip(revenueList)
    revenue = 0
    for choice in choices:
        revenue += revenueList[choice]
        
    return revenue

In [6]:
def seatsSold(choices):
    return len(choices) - np.sum(np.array(choices) == 9)

In [7]:
lam_business = lambda t: 1.2*np.sin(t*np.pi/180.0)
lam_max_business = 1.2
lam_leisure = lambda t: 0.6*t/179
lam_max_leisure = 0.6
lam_economy = lambda t: 0.8*(1 + np.sin(t*np.pi/180.0 + np.pi))
lam_max_economy = 0.8

def non_homogeneous_poisson_process(lam_t, lam_max, T):
    arrivals = []
    t = exponential_rng(lam_max)
    while t <= T:
        if np.random.rand() < lam_t(t)/lam_max:
            arrivals.append(t)
        t += exponential_rng(lam_max)
    return arrivals

In [13]:
class Passenger:
    """ Generic event.
    
    Attributes:
        time (float): Event time.
        preferences (array): fare product preference weights.
    """
    
    def __init__(self, time):
        self.time = time
        self.preferences = np.zeros(10)
        
    def makeChoice(self,availabilities):
        
        probabilities = self.preferences*(availabilities > 0)
        probabilities = probabilities/np.sum(probabilities)
        
        cumsum = np.cumsum(probabilities)
        
        r = np.random.rand()
        
        choice = np.min(np.where(r < cumsum ))
        
        return choice
    
    def makeIdealChoice(self):
        
        probabilities = self.preferences
        probabilities = probabilities/np.sum(probabilities)
        
        cumsum = np.cumsum(probabilities)
        
        r = np.random.rand()
        
        choice = np.min(np.where(r < cumsum ))
        
        return choice
        
        
        
class Business(Passenger):

    def __init__(self, time):
        super().__init__(time)
        self.preferences = np.array([11, 15, 18, 20, 19, 15, 12, 11, 13, 7])
        
    def makeChoice(self,availabilities):
        return Passenger.makeChoice(self,availabilities)
    
    def makeIdealChoice(self):
        return Passenger.makeIdealChoice(self)
        
class Leisure(Passenger):

    def __init__(self, time):
        super().__init__(time)
        self.preferences = np.array([8, 9, 11, 12, 14, 15, 16, 18, 20, 7])
        
    def makeChoice(self,availabilities):
        return Passenger.makeChoice(self,availabilities)
    
    def makeIdealChoice(self):
        return Passenger.makeIdealChoice(self)
        
class Economy(Passenger):

    def __init__(self, time):
        super().__init__(time)
        self.preferences = np.array([1, 5, 8, 10, 11, 12, 13, 15, 20, 7])
        
    def makeChoice(self,availabilities):
        return Passenger.makeChoice(self,availabilities)
    
    def makeIdealChoice(self):
        return Passenger.makeIdealChoice(self)

In [14]:
class Scenario:
    """ Road scenario
    
    Attributes:
        planning_horizon (int): days.
        t (int): early sales discount
        lam_business (float):
        lam_leisure (float):
        lam_economy (float):
    """
    
    def __init__(self, 
                 planning_horizon=180,
                 t=21,
                 lam_business=1.2,
                 lam_leisure=0.6,
                 lam_economy=0.8,
                 flag=0 # 1 if we want to offer 1 product at a time 
                ):
        self.planning_horizon = planning_horizon
        self.t = t
        self.lam_business = lam_business
        self.lam_leisure = lam_leisure
        self.lam_economy = lam_economy
        self.flag = flag

In [32]:
def simulate(scenario):
    """ Implements the simulation procedure.
    
    Keywords:
        scenario (Scenario): Road scenario.
    
    Returns:
        times (list): Event times. 
        queues (list): Queue length over time. 
    """
    
    #Initialise the simulation
    t = 179
    events = []
    
    fare_offered = -1
    
    if scenario.flag:
        availabilities = np.zeros([10])
        fare_offered = 8 # The last element is "no purchase" so it is never offered
        availabilities[fare_offered] = 20 # Start from fare I and offer one at a time up to fare A
    else:
        availabilities = 20*np.ones([10])
        
    
    
    availabilities[-1] = np.Inf # We can offer infinitely many non-purchases
    revenues = 0
    
    #Trigger the first events
    
    # Generate customer arrivals
    business_arrival_process = non_homogeneous_poisson_process(lam_business, lam_max_business, t)
    for i in business_arrival_process:
        events.append(Business(i))
    
    leisure_arrival_process = non_homogeneous_poisson_process(lam_leisure, lam_max_leisure, t)
    for i in leisure_arrival_process:
        events.append(Leisure(i))
    
    economy_arrival_process = non_homogeneous_poisson_process(lam_economy, lam_max_economy, t)
    for i in economy_arrival_process:
        events.append(Economy(i))
        
    events.sort(key=lambda event: event.time)
    
    '''
    termination = Termination(scenario.demand_duration)
    events = [generation, termination]
    '''
    choices = []
    idealChoices = []
    
    #Main loop
    while len(events):
        
        t = events[0].time
        
        e = events[0]
        
        choice = e.makeChoice(availabilities)
        choices.append(choice)
        
        idealChoice = e.makeIdealChoice()
        idealChoices.append(idealChoice)
        
        availabilities[choice] -= 1
        
        
        if scenario.flag:
            if availabilities[fare_offered] == 0 and fare_offered !=0:
                fare_offered -= 1
                availabilities[fare_offered] = 20
                
        if t < 21:
            availabilities[3] = 0
            availabilities[7] = 0
            
            if fare_offered == 3 or fare_offered == 7:
                fare_offered -= 1
                
                if scenario.flag:
                    availabilities[fare_offered] = 20
            
        
            
        events.pop(0)
    
        
    return choices, idealChoices

In [33]:
choices,idealChoices = simulate(Scenario(1))

In [34]:
print(totalRevenue(choices))

95000


In [35]:
print(seatsSold(choices))

140


In [36]:
seats_available=180

In [37]:
print(idealChoices)

[8, 9, 8, 3, 6, 3, 7, 6, 8, 8, 2, 6, 5, 8, 3, 4, 3, 4, 3, 6, 4, 7, 7, 9, 8, 8, 8, 1, 2, 0, 3, 6, 4, 0, 7, 4, 1, 5, 6, 5, 8, 1, 5, 8, 1, 2, 2, 5, 4, 8, 8, 9, 0, 4, 6, 0, 1, 2, 5, 9, 6, 3, 3, 6, 6, 1, 0, 4, 5, 8, 0, 5, 2, 0, 5, 5, 2, 0, 0, 0, 2, 3, 4, 7, 1, 6, 6, 2, 1, 2, 4, 8, 8, 7, 8, 2, 1, 6, 6, 1, 6, 2, 7, 1, 4, 0, 5, 0, 6, 9, 0, 8, 0, 5, 5, 8, 1, 3, 0, 6, 4, 6, 1, 5, 8, 0, 0, 3, 4, 6, 2, 7, 4, 4, 2, 0, 5, 3, 3, 8, 8, 8, 2, 9, 8, 8, 1, 6, 0, 3, 1, 1, 4, 6, 2, 2, 2, 1, 8, 7, 8, 1, 4, 8, 1, 4, 1, 4, 5, 5, 4, 6, 1, 2, 4, 7, 2, 7, 2, 5, 6, 2, 1, 6, 8, 2, 5, 2, 7, 5, 3, 1, 9, 1, 6, 5, 2, 9, 8, 5, 3, 1, 2, 4, 2, 5, 6, 1, 6, 8, 6, 7, 3, 8, 7, 7, 1, 8, 6, 5, 6, 9, 9, 9, 5, 6, 2, 2, 7, 0, 8, 3, 6, 3, 8, 2, 3, 2, 4, 7, 7, 2, 6, 2, 3, 3, 1]


In [38]:
print(totalRevenue(idealChoices))

168200


In [39]:
print(len (idealChoices))

247


In [40]:
print(len (choices))

247


In [42]:
print (seatsSold(idealChoices))

236
